# MaBoSS Course

First we import MaBoSS

In [1]:
import maboss

This notebook has been executed using the docker image `colomoto/colomoto-docker:2022-05-01`

https://journals.plos.org/ploscompbiol/article/authors?id=10.1371/journal.pcbi.1004571

To elucidate the role of individual mutations or their combinations affecting the metastatic development, a logical model has been constructed that recapitulates published experimental results of known gene perturbations on local invasion and migration processes, and predict the effect of not yet experimentally assessed mutations. The model has been validated using experimental data on transcriptome dynamics following TGF-β-dependent induction of Epithelial to Mesenchymal Transition in lung cancer cell lines

### Loading the model

In [2]:
help(maboss.load)

Help on function load in module maboss.gsparser:

load(bnd_filename, *cfg_filenames, **extra_args)
    Loads a network from a MaBoSS format file.
    
    :param str bnd_filename: Network file
    :param str cfg_filename: Configuraton file
    :keyword str simulation_name: name of the returned :py:class:`.Simulation` object
    :rtype: :py:class:`.Simulation`



In [3]:
model_invasion = maboss.load(COHEN_NETWORK_FILE, COHEN_CONFIG_FILE)

NameError: name 'COHEN_NETWORK_FILE' is not defined

### Running the simulation

In [ ]:
help(model_invasion)

In [ ]:
help(model_invasion.run)

In [ ]:
res_invasion = model_invasion.run()

### Visualising the steady states distribution

In [ ]:
help(res_invasion)

In [ ]:
res_invasion.plot_piechart()

### Visualising the trajectory of steady states distribution

In [ ]:
res_invasion.plot_trajectory()

### Visualizing the trajectory of individual nodes

In [ ]:
res_invasion.plot_node_trajectory()

### Getting the trajectories as a pandas dataframe

In [ ]:
table = res_invasion.get_states_probtraj()
table

In [ ]:
n_ap_n_inv = table["<nil>"]+table["CellCycleArrest"]
n_ap_n_inv[n_ap_n_inv.index <= 10].plot()

### Simulating the model without ECM nor DNA damage

In [ ]:
no_ecm_no_dd = model_invasion.copy()

In [ ]:
help(no_ecm_no_dd.network.set_istate)

In [ ]:
no_ecm_no_dd.network.set_istate('ECMicroenv', [1, 0])
no_ecm_no_dd.network.set_istate('DNAdamage', [1, 0])

In [ ]:
res_no_ecm_no_dd = no_ecm_no_dd.run()

In [ ]:
res_no_ecm_no_dd.plot_piechart()

### Simulating with ECM but no DNADamage

In [ ]:
ecm_no_dd = model_invasion.copy()

In [ ]:
ecm_no_dd.network.set_istate('ECMicroenv', [0, 1])
ecm_no_dd.network.set_istate('DNAdamage', [1, 0])

In [ ]:
res_ecm_no_dd = ecm_no_dd.run()

In [ ]:
res_ecm_no_dd.plot_piechart()

### Simulating with ECM and DNA damage

In [ ]:
ecm_dd = model_invasion.copy()

In [ ]:
ecm_dd.network.set_istate('ECMicroenv', [0, 1])
ecm_dd.network.set_istate('DNAdamage', [0, 1])

In [ ]:
res_ecm_dd = ecm_dd.run()

In [ ]:
res_ecm_dd.plot_piechart()

### Simulating Notch++, p53--

In [ ]:
mutant_invasion = model_invasion.copy()

In [ ]:
help(mutant_invasion.mutate)

In [ ]:
print(mutant_invasion.network.names)

In [ ]:
mutant_invasion.mutate('NICD', 'ON')
mutant_invasion.mutate('p53', 'OFF')

In [ ]:
res_mutant_invasion = mutant_invasion.run()

In [ ]:
res_mutant_invasion.plot_piechart()

## Montagud Prostate Cancer

https://elifesciences.org/articles/72626

### Loading the model

In [ ]:
model_prostate = maboss.load("models/Montagud2022_Prostate_Cancer.bnd", "models/Montagud2022_Prostate_Cancer.cfg")

### Run the default simulation

In [ ]:
res_prostate = model_prostate.run()

### Print results

In [ ]:
res_prostate.plot_trajectory()

### Printing the list of nodes

In [ ]:
print(model_prostate.network.names)

### Simulating the WT 

These healthy cells mostly exhibit quiescence (neither proliferation nor apoptosis) in the absence of any input (Figure 3A)

In [ ]:
for node in model_prostate.network.keys():
    model_prostate.network.set_istate(node, [1, 0])

In [ ]:
res_prostate_zero = model_prostate.run()
res_prostate_zero.plot_piechart()

### Simulating the growth condition

When Nutrients and growth factors (EGF or FGF) are present, Proliferation is activated (Figure 3B).

In [ ]:
prostate_growth = model_prostate.copy()

In [ ]:
prostate_growth.network.set_istate('EGF', [0, 1])
prostate_growth.network.set_istate('Nutrients', [0, 1])

In [ ]:
res_prostate_growth = prostate_growth.run()

In [ ]:
res_prostate_growth.plot_piechart()

### Simulating the apoptosis phenotype

The model was simulated with Carcinogen, Androgen, TNFalpha, Acidosis, and Hypoxia ON.

In [ ]:
prostate_death = model_prostate.copy()

In [ ]:
prostate_death.network.set_istate('Carcinogen', [0, 1])
prostate_death.network.set_istate('Androgen', [0, 1])
prostate_death.network.set_istate('TNFalpha', [0, 1])
prostate_death.network.set_istate('Acidosis', [0, 1])
prostate_death.network.set_istate('Hypoxia', [0, 1])

In [ ]:
res_prostate_death = prostate_death.run()

In [ ]:
res_prostate_death.plot_piechart()

### Simulating MYC_MAX mutant on proliferative phenotype

In [ ]:
mutant_prostate_growth = prostate_growth.copy()

In [ ]:
mutant_prostate_growth.mutate("MYC_MAX", "OFF")

In [ ]:
res_mutant_prostate_growth = mutant_prostate_growth.run()

In [ ]:
res_mutant_prostate_growth.plot_piechart()

### Checking multiple single node mutants 

In [ ]:
from maboss.pipelines import simulate_single_mutants, filter_sensitivity

In [ ]:
help(simulate_single_mutants)

In [ ]:
simulations = simulate_single_mutants(prostate_growth, ["AKT", "AR", "EGFR", "ERK", "MYC_MAX", "ROS"], "OFF")

In [ ]:
for simulation, result in simulations.items():
    result.plot_piechart()
    plt.title(simulation)

### Filter set of results which are not proliferative

In [ ]:
help(filter_sensitivity)

In [ ]:
 = filter_sensitivity(simulations, state='Proliferation', maximum=0)

In [ ]:
for simulation, result in res_filtered.items():
    result.plot_piechart()
    plt.title(simulation)